<center> <h2> DS 3000 - Spring 2019</h2> </center>
<center> <h3> DS Report </h3> </center>


<center> <h3> Influence of US Demographic Factors on Personal Income </h3> </center>
<center><h4>Aaditya Watwe, Arian Gokhale, Lily Olson, Alex Shaw</h4></center>


<hr style="height:2px; border:none; color:black; background-color:black;">

#### Executive Summary:

Add your summary here (100-150 words)

Provide a brief summary of your project. After reading this executive summary, your readers should have a rough understanding of what you did in this project. You can think of this summary in terms of the four sections of the report and write 1-2 sentences describing each section.



<hr style="height:2px; border:none; color:black; background-color:black;">

## Outline
1. <a href='#1'>INTRODUCTION</a>
2. <a href='#2'>METHOD</a>
3. <a href='#3'>RESULTS</a>
4. <a href='#4'>DISCUSSION</a>

<a id="1"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 1. INTRODUCTION

Our project decided to tackle understanding how different demographics can affect the personal income of individuals. We believe that there are some data visualizations tackling this issue, but many are highly politicized or hard to interpet. This has led to a lack of understanding on this topic by the general pulic. The topic of our project is determining to what degree an income gap exists and along which demographic borders does it fall. Some of our learning objectives include analyzing the gender-based income gap and the model minority theories. We would also like to analyze which demographic factors are the most significant in influencing one's income.

This is an important problem to tackle, because without an unbiased understanding of the problem, we cannot create well informed public policy. The lack of concrete evidence towards either side of the political divide over this issue creates more misinformation and angst rather than approaching a solution. The insights derived from this project could show where the primary causes of income gaps are. Thus the problem is no longer the “income gap”, rather the problem will more clearly be identified (i.e. If the primary gap found is the lack of X race in Y industry, future studies could examine the cause of that phenomenon). There has been some research done in this area. A report titled Income and Wealth in the United States by the Peter G. Peterson Foundation found that incomes varied considerably between the coasts and the midwest and the south of the United States. Furthermore, income varied based on race/ethnicity, educational attainment, and gender. However, the analysis fozues on the median income levels, and does not produce any correlataional analysis (1). Other projects have looked into how the middle class has shrunk over time in America, but not dived into specifics of demographic data (2). 

Given the aforementioned problem and its importance, we set out to tackle the following questions:
* To what degree do factors including gender, race, family wealth, geographic area, and occupational industry influence one's income?
* To what extent does the gender-based income gap exist and are there deviations for different wealth brackets, races, or geographic areas?

<a id="2"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 2. METHOD

### 2.1. Data Acquisition

* Describe where you obtained your data. Provide a link to the original source. 
* If you scraped your data, include your code as a separate script file.
* Your data should be stored in an online repository (e.g., GitHub) and your code should retrieve your data from that online resource. You can read csv files from the Web in the same way that you read files from local drive.
* Describe the dataset and variables. What do variables represent?


### 2.2. Variables
* For your hypotheses, what are your IVs and DVs?
* For your predictive models, what are your features and target variables?


### 2.3. Data Analysis
* Specifically describe your predictive model. What outcome variable are you going to predict from what feature variables?
* Describe whether this is a supervised or unsupervised learning problem. Also identify the sub-category of the learning task (e.g. classification).
* What machine learning algorithms are you going to use? Why?

<a id="3"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 3. RESULTS

### 3.1. Data Wrangling
* Perform simple data cleaning (delete extra columns, deal with NA values, etc.)
* Perform data wrangling to extract your features and target values (e.g., grouping your dataframe by columns, applying functions to format dataframes, etc.)
* Preprocess your variables (e.g., scaling/transforming feature variables to normalize them)
* Feature extraction (dummy variables, new features from existing features, etc.)
* Use one feature selection technique to select a subset of your original features


### 3.2. Data Exploration
* Generate appropriate data visualizations for your key variables identified in the previous section
* You should have at least three visualizations (and at least two different visualization types)
* For each visualization provide an explanation regarding the variables involved and an interpretation of the graph.
* If you are using Plotly, insert your visualizations as images as well (upload the graph images to an online source, e.g. github, and link those in Jupyter Notebook)


# Create Regression Models to Predict Income
## **A Notebook for Using Demographic Data to Predict Income/Wage**

#### Import the data from pickles, and separate into the features/targets

In [1]:
import pandas as pd
import numpy as np

In [2]:
from datetime import datetime

def log(message):
    print(datetime.now().strftime("%H:%M:%S -"), message)
    
def printnow():
    print(datetime.now().strftime("Current time: %H:%M:%S"))

In [3]:
log('Reading data')
us_personal = pd.read_pickle('preprocessed_data/onehot_data.zip')
log('Done\n')

22:42:00 - Reading data
22:42:16 - Done



In [4]:
from sklearn.preprocessing import RobustScaler # a scaler to overcome outliers from the data
from sklearn.linear_model import ElasticNet, SGDRegressor # baseline regression models
from keras import Sequential # all relevant imports for Keras (TensorFlow based neural net)
from keras.layers import Dense, LeakyReLU, ELU
from keras.wrappers.scikit_learn import KerasRegressor

# metrics to exaluate the performance of our regressors
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split

Using TensorFlow backend.


In [5]:
# Separate the data into three groups: incomes or wages (targets), and features (all other columns)
# and return a corresponding dictionary with the three sets
def features_targets(df):
    features = df.drop(columns=['PINCP', 'WAGP'])
    incomes = df['PINCP']
    wages = df['WAGP']
    return {'features':features, 'incomes':incomes, 'wages':wages}

# create a dictionary representing the three sets and their features/targets
log('Splitting Train/Test Data')
train, test = train_test_split(us_personal, test_size=0.2)
dataset = {
    'train':features_targets(train),
    'test':features_targets(test)}


# Initialize the RobustScaler on the training data (fit before training each regression model)
log('Scaling Train/Test Features')
robust_scaler = RobustScaler().fit(dataset['train']['features'])
dataset['train']['features'] = robust_scaler.transform(dataset['train']['features'])
dataset['test']['features'] = robust_scaler.transform(dataset['test']['features'])
log('Done\n')

22:42:19 - Splitting Train/Test Data
22:42:40 - Scaling Train/Test Features
22:43:06 - Done



In [6]:
# a function to fit, test, and print results of a given estimator for a given target column
def fit_test_print(estimator, estimator_name, target, target_name, grid=1):
    log(f'Training {estimator_name}')
    estimator.fit(dataset['train']['features'], dataset['train'][target].array)
    y_pred_train = estimator.predict(dataset['train']['features'])
    y_pred_test = estimator.predict(dataset['test']['features'])
    log('Done\n')
    
    print(f"Results for {target_name} with {estimator_name}")
    if grid == 1: print(f"- Best parameters: {estimator.best_params_}")

    print(f"- Training Set")
    print(f"\tMean Squared Error: {mean_squared_error(dataset['train'][target], y_pred_train)}")
    print(f"\tMedian Absolute Error: {median_absolute_error(dataset['train'][target], y_pred_train)}")
    print(f"\tr-Squared: {r2_score(dataset['train'][target], y_pred_train)}")

    print(f"- Test Set")
    print(f"\tMean Squared Error: {mean_squared_error(dataset['test'][target], y_pred_test)}")
    print(f"\tMedian Absolute Error: {median_absolute_error(dataset['test'][target], y_pred_test)}")
    print(f"\tr-Squared: {r2_score(dataset['test'][target], y_pred_test)}")

## Linear Regression
Implemented with ElasticNet, which combines L1 and L2 regularization

In [9]:
linear_model_params = {
    'alpha':[0.01, 0.1, 1, 10],
    'l1_ratio':[0.25, 0.50, 0.75, 1]
}

linear_model_i = GridSearchCV(estimator=ElasticNet(), param_grid=linear_model_params, 
                            scoring=['neg_median_absolute_error', 'r2'], refit='r2', cv=4)
linear_model_w = GridSearchCV(estimator=ElasticNet(), param_grid=linear_model_params, 
                            scoring=['neg_median_absolute_error', 'r2'], refit='r2', cv=4)

In [8]:
fit_test_print(linear_model_i, 'Linear Model L1/L2 Regularized', 'incomes', 'Income')

16:14:08 - Training Linear Model L1/L2 Regularized
16:26:53 - Done

Results for Income with Linear Model L1/L2 Regularized
- Best parameters: {'alpha': 10, 'l1_ratio': 0.6}
- Training Set
	Mean Squared Error: 3274596194.857993
	Median Absolute Error: 22929.83070571476
	r-Squared: 0.11891055597779776
- Test Set
	Mean Squared Error: 3281850782.70632
	Median Absolute Error: 22924.74442348032
	r-Squared: 0.11853319389596961


In [10]:
fit_test_print(linear_model_w, 'Linear Model L1/L2 Regularized', 'wages', 'Wage')

16:29:04 - Training Linear Model L1/L2 Regularized
16:37:42 - Done

Results for Wage with Linear Model L1/L2 Regularized
- Best parameters: {'alpha': 10, 'l1_ratio': 0.6}
- Training Set
	Mean Squared Error: 2351956604.698796
	Median Absolute Error: 17558.869250567495
	r-Squared: 0.15201561556297183
- Test Set
	Mean Squared Error: 2333282440.602135
	Median Absolute Error: 17562.433428652887
	r-Squared: 0.15276740461628124


## SGD Regression
Loss set to Huber to be more robust to outliers in income and wage

In [7]:
sgd_model_params = {
    'alpha':[0.000001, 0.00001, 0.0001, 0.001]
}

sgd_model_i = GridSearchCV(estimator=SGDRegressor(penalty='elasticnet', learning_rate='adaptive'), param_grid=sgd_model_params, cv=4)
sgd_model_w = GridSearchCV(estimator=SGDRegressor(penalty='elasticnet', learning_rate='adaptive'), param_grid=sgd_model_params, cv=4)

In [8]:
fit_test_print(sgd_model_i, 'SGD Model L1/L2 Regularized', 'incomes', 'Income')

22:43:06 - Training SGD Model L1/L2 Regularized
23:35:27 - Done

Results for Income with SGD Model L1/L2 Regularized
- Best parameters: {'alpha': 0.0001}
- Training Set
	Mean Squared Error: 2313730116.8329206
	Median Absolute Error: 12513.364387014095
	r-Squared: 0.3787176688793734
- Test Set
	Mean Squared Error: 2288257245.7425137
	Median Absolute Error: 12522.970271169928
	r-Squared: 0.380348437367074


In [10]:
fit_test_print(sgd_model_w, 'Logistic Model L1/L2 Regularized', 'wages', 'Wage')

23:35:28 - Training Logistic Model L1/L2 Regularized
00:21:39 - Done

Results for Wage with Logistic Model L1/L2 Regularized
- Best parameters: {'alpha': 0.0001}
- Training Set
	Mean Squared Error: 1565194628.2093012
	Median Absolute Error: 9285.146097658673
	r-Squared: 0.4363179455367291
- Test Set
	Mean Squared Error: 1536207636.0796754
	Median Absolute Error: 9283.257766323872
	r-Squared: 0.4396291032552606


## Keras Neural Net Regression
Implemented with feed-forward dense layers 

In [13]:
# a function to initialize the model
def init_keras():
    model = Sequential()
    model.add(Dense(200, input_dim=151))
    model.add(ELU())
    model.add(Dense(200))
    model.add(LeakyReLU())
    model.add(Dense(50))
    model.add(LeakyReLU())
    model.add(Dense(20))
    model.add(LeakyReLU())
    model.add(Dense(1))
    model.compile(loss='huber_loss', optimizer='adam', metrics=['mean_absolute_error', 'mean_squared_error'])
    return model

nn_model_i = KerasRegressor(build_fn=init_keras, epochs=50, batch_size=32, verbose=1)
nn_model_w = KerasRegressor(build_fn=init_keras, epochs=50, batch_size=32, verbose=1)

In [14]:
fit_test_print(nn_model_i, 'Keras Neural Net', 'incomes', 'Income', grid=0)

00:29:41 - Training Keras Neural Net
Epoch 1/50
2571631/2571631 [==============================] - 176s 69us/step - loss: 17270.2945 - mean_absolute_error: 17270.8086 - cosine_proximity: 0.7193
Epoch 2/50
2571631/2571631 [==============================] - 201s 78us/step - loss: 16832.5715 - mean_absolute_error: 16832.9355 - cosine_proximity: 0.7194
Epoch 3/50
2571631/2571631 [==============================] - 198s 77us/step - loss: 16707.9343 - mean_absolute_error: 16708.3223 - cosine_proximity: 0.7192
Epoch 4/50
2571631/2571631 [==============================] - 198s 77us/step - loss: 16629.6131 - mean_absolute_error: 16629.9766 - cosine_proximity: 0.7189
Epoch 5/50
2571631/2571631 [==============================] - 198s 77us/step - loss: 16573.3254 - mean_absolute_error: 16573.9473 - cosine_proximity: 0.7187
Epoch 6/50
2571631/2571631 [==============================] - 203s 79us/step - loss: 16533.2628 - mean_absolute_error: 16533.7871 - cosine_proximity: 0.7187
Epoch 7/50
2571631/25

In [15]:
fit_test_print(nn_model_w, 'Keras Neural Net', 'wages', 'Wage', grid=0)

03:11:26 - Training Keras Neural Net
Epoch 1/50
2571631/2571631 [==============================] - 186s 72us/step - loss: 11071.2180 - mean_absolute_error: 11071.6953 - cosine_proximity: 0.4865
Epoch 2/50
2571631/2571631 [==============================] - 186s 72us/step - loss: 10731.7060 - mean_absolute_error: 10732.1641 - cosine_proximity: 0.4863
Epoch 3/50
2571631/2571631 [==============================] - 186s 72us/step - loss: 10654.8675 - mean_absolute_error: 10655.3721 - cosine_proximity: 0.4861
Epoch 4/50
2571631/2571631 [==============================] - 186s 72us/step - loss: 10607.6285 - mean_absolute_error: 10608.1494 - cosine_proximity: 0.4860
Epoch 5/50
2571631/2571631 [==============================] - 186s 72us/step - loss: 10567.5109 - mean_absolute_error: 10567.9746 - cosine_proximity: 0.4860
Epoch 6/50
2571631/2571631 [==============================] - 186s 72us/step - loss: 10537.6260 - mean_absolute_error: 10538.1807 - cosine_proximity: 0.4860
Epoch 7/50
2571631/25

### 3.3. Model Construction
* If you proposed hypotheses, conduct your hypothesis tests
* For your machine learning question(s), split data into training, validation, and testing sets (or use cross-validation)
* Apply machine learning algorithms (apply at least three algorithms)
* Train your algorithms

### 3.4. Model Evaluation
* Evaluate the performance of your algorithms on appropriate evaluation metrics, using your validation set
* Interpret your results from multiple models (and hypothesis tests, if any)

### 3.5. Model Optimization
* Tune your models using appropriate hyperparameters
* Explain why you are doing this (e.g., to avoid overfitting, etc.)

### 3.6. Model Testing
* Test your tuned algorithms using your testing set

<a id="4"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

## 4. DISCUSSION
* Provide a summary of the steps you took to analyze your data and test your predictive model
* Intepret your findings from 3.4., 3.5, and 3.6
    * Which algorithms did you compare?
    * Which algorithm(s) revealed best performance?
    * Which algorithm(s) should be used for your predictive model?
* If you tested hypotheses, interpret the results. What does it mean to have significant/non-significant differences with regards to your data?


* End this section with a conclusion paragraph containing some pointers for future work 
    *(e.g., get more data, perform another analysis, etc.)

<a id="5"></a>
<hr style="height:2px; border:none; color:black; background-color:black;">

### CONTRIBUTIONS
* Describe each team member's contributions to the report (who did what in each section)
* Remember this is a team effort!
* Each member of your team will provide peer evaluation of other team members. Your final grade on the project will be based on those peer evaluations. 